In [4]:
import pandas as pd
import numpy as np
import copy
import torch

from config import config
from datasets.epfl_sk30 import EPFLSK30Dataset
from torch.utils.data import DataLoader
from datasets.epfl_sk30_eval import EPFLSK30Eval

In [22]:
# Configuration
results_keys = ['#2', '#4', '#8', '#10', '#14', '#18', '#22', '#25']

# Joint names for EPFL-SK30 dataset
joint_names = [
    'Root',
    'LeftHead',
    'RightHead',
    'LeftBody',
    'RightBody',
    'LeftShoulder',
    'RightShoulder',
    'LeftArm',
    'RightArm',
    'LeftForearm',
    'RightForearm',
    'LeftHip',
    'RightHip',
    'LeftKnee',
    'RightKnee',
    'LeftFoot',
    'RightFoot'
]

In [6]:
def compute_zero_velocity_baseline(dataloader, target_length, config):
    """
    Compute zero velocity baseline by repeating the last observed frame.
    
    Args:
        dataloader: DataLoader for the dataset
        target_length: Number of future frames to predict
        config: Configuration object
        
    Returns:
        ret: Dictionary with average MPJPE per timestep
        ret_per_joint: Dictionary with MPJPE per joint per timestep
    """
    m_p3d_h36_per_joint = np.zeros([target_length, 17])  # 17 joints
    num_samples = 0
    
    for (motion_t, motion_t_1) in dataloader:
        b, n, c, _ = motion_t.shape
        num_samples += b
        
        # motion_t[:, 49, :, :] has shape (b, 17, 3) - last observed frame
        # Expand to (b, target_length, 17, 3)
        motion_pred = motion_t[:, config.motion.epfl_input_length - 1, :, :].unsqueeze(1).repeat(1, target_length, 1, 1)
        motion_gt = motion_t_1
        
        # Calculate MPJPE per joint for each timestep
        # dim=3: L2 norm over (x,y,z) -> (b, target_length, 17)
        # dim=0: sum over batch -> (target_length, 17)
        mpjpe_p3d_h36_per_joint = torch.sum(torch.norm(motion_pred*1000 - motion_gt*1000, dim=3), dim=0)
        m_p3d_h36_per_joint += mpjpe_p3d_h36_per_joint.cpu().numpy()
        
    m_p3d_h36_per_joint = m_p3d_h36_per_joint / num_samples
    m_p3d_h36 = m_p3d_h36_per_joint.mean(axis=1)
    
    titles = np.array(range(target_length)) + 1
    ret = {}
    ret_per_joint = {}
    for j in range(target_length):
        ret["#{:d}".format(titles[j])] = m_p3d_h36[j]
        ret_per_joint["#{:d}".format(titles[j])] = m_p3d_h36_per_joint[j]
    
    return ret, ret_per_joint


In [7]:
eval_dataset = EPFLSK30Eval(config, 'test')
eval_dataloader = DataLoader(eval_dataset, batch_size=256,
                        num_workers=16, drop_last=True,
                        sampler=None, shuffle=False, pin_memory=True)
print(f"Length of test dataloader: {len(eval_dataloader)}")

Length of test dataloader: 2099


In [8]:
# To evaluate the training set in up to 25 frames
config.motion.epfl_target_length_train = 25

dataset = EPFLSK30Dataset(config, 'train', config.data_aug)
dataloader = DataLoader(dataset, batch_size=256,
                        num_workers=16, drop_last=False,
                        sampler=None, shuffle=False, pin_memory=True)
print(f"Length of train dataloader: {len(dataloader)}")

Length of train dataloader: 4352


In [ ]:
# Compute zero velocity baseline for training set
ret_train, ret_per_joint_train = compute_zero_velocity_baseline(dataloader, config.motion.epfl_target_length_train, config)

In [ ]:
print("Zero Velocity Baseline Results for Training Set:")
print([key for key in results_keys])
print([round(ret_train[key], 1) for key in results_keys])

print("\n" + "="*60)
print("Per-Joint Errors for each timestep:")
print("="*60)
for key in results_keys:
    print(f"\nTimestep {key}:")
    per_joint_errors = ret_per_joint_train[key]
    for joint_idx in range(len(per_joint_errors)):
        print(f"  Joint {joint_idx:2d} ({joint_names[joint_idx]:13s}): {per_joint_errors[joint_idx]:.3f} mm")
    print(f"  Average: {ret_train[key]:.3f} mm")

Zero Velocity Baseline Results for Training Set:
['#2', '#4', '#8', '#10', '#14', '#18', '#22', '#25']
[20.3, 38.4, 70.3, 84.8, 111.1, 134.7, 156.1, 170.9]

Per-Joint Errors for each timestep:

Timestep #2:
  Joint  0 (Root): 20.500 mm
  Joint  1 (LeftHead): 20.222 mm
  Joint  2 (RightHead): 20.248 mm
  Joint  3 (LeftBody): 19.747 mm
  Joint  4 (RightBody): 19.787 mm
  Joint  5 (LeftShoulder): 20.032 mm
  Joint  6 (RightShoulder): 21.243 mm
  Joint  7 (LeftArm): 23.361 mm
  Joint  8 (RightArm): 28.295 mm
  Joint  9 (LeftForearm): 27.556 mm
  Joint 10 (RightForearm): 34.358 mm
  Joint 11 (LeftHip): 16.609 mm
  Joint 12 (RightHip): 17.130 mm
  Joint 13 (LeftKnee): 14.728 mm
  Joint 14 (RightKnee): 15.468 mm
  Joint 15 (LeftFoot): 12.909 mm
  Joint 16 (RightFoot): 13.750 mm
  Average: 20.350 mm

Timestep #4:
  Joint  0 (Root): 39.346 mm
  Joint  1 (LeftHead): 39.002 mm
  Joint  2 (RightHead): 39.023 mm
  Joint  3 (LeftBody): 37.711 mm
  Joint  4 (RightBody): 37.833 mm
  Joint  5 (LeftShou

In [ ]:
# Compute zero velocity baseline for test set
ret_eval, ret_per_joint_eval = compute_zero_velocity_baseline(eval_dataloader, config.motion.epfl_target_length_eval, config)

In [21]:
print("Zero Velocity Baseline Results for Test Set:")
print([key for key in results_keys])
print([round(ret_eval[key], 1) for key in results_keys])

print("\n" + "="*60)
print("Per-Joint Errors for each timestep:")
print("="*60)
for key in results_keys:
    print(f"\nTimestep {key}:")
    per_joint_errors = ret_per_joint_eval[key]
    for joint_idx in range(len(per_joint_errors)):
        print(f"  Joint {joint_idx:2d} ({joint_names[joint_idx]:13s}): {per_joint_errors[joint_idx]:.3f} mm")
    print(f"  Average: {ret_eval[key]:.3f} mm")

Zero Velocity Baseline Results for Test Set:
['#2', '#4', '#8', '#10', '#14', '#18', '#22', '#25']
[18.8, 35.4, 64.4, 77.4, 101.1, 122.2, 141.2, 154.4]

Per-Joint Errors for each timestep:

Timestep #2:
  Joint  0 (Root         ): 18.350 mm
  Joint  1 (LeftHead     ): 18.030 mm
  Joint  2 (RightHead    ): 18.095 mm
  Joint  3 (LeftBody     ): 17.686 mm
  Joint  4 (RightBody    ): 17.835 mm
  Joint  5 (LeftShoulder ): 17.936 mm
  Joint  6 (RightShoulder): 19.405 mm
  Joint  7 (LeftArm      ): 20.845 mm
  Joint  8 (RightArm     ): 26.670 mm
  Joint  9 (LeftForearm  ): 24.766 mm
  Joint 10 (RightForearm ): 33.034 mm
  Joint 11 (LeftHip      ): 15.725 mm
  Joint 12 (RightHip     ): 16.274 mm
  Joint 13 (LeftKnee     ): 13.911 mm
  Joint 14 (RightKnee    ): 14.891 mm
  Joint 15 (LeftFoot     ): 12.399 mm
  Joint 16 (RightFoot    ): 13.405 mm
  Average: 18.780 mm

Timestep #4:
  Joint  0 (Root         ): 35.129 mm
  Joint  1 (LeftHead     ): 34.663 mm
  Joint  2 (RightHead    ): 34.775 mm
  